In [32]:
import pandas as pd

In [33]:
df_FUND = pd.read_csv("data/PB_INFRAESTRUTURA_FUND")
df_MED = pd.read_csv("data/PB_INFRAESTRUTURA_MED")

In [34]:
df_FUND.rename(columns={"Unnamed: 0": "ID_ESCOLA"}, inplace=True)

In [35]:
df_MED.rename(columns={"Unnamed: 0": "ID_ESCOLA"}, inplace=True)

In [36]:
df_FUND

,ID_ESCOLA,IN_INTERNET_ALUNOS,IN_BANDA_LARGA,IN_REFEITORIO,IN_ESPACO_ATIVIDADE,IN_ACESSO_INTERNET_COMPUTADOR,TP_DEPENDENCIA,QT_MAT_FUND,TP_LOCALIZACAO,QT_PROF_PSICOLOGO,QT_PROF_PEDAGOGIA,QT_TRANSP_PUBLICO,QT_DOC_FUND
0,60497,1.0,1.0,0.0,0.0,0.0,3,675.0,1,0.0,3.0,207.0,42.0
1,60498,NaN,NaN,NaN,NaN,NaN,3,NaN,1,NaN,NaN,NaN,NaN
2,60499,1.0,1.0,1.0,1.0,0.0,2,0.0,1,0.0,1.0,20.0,0.0
3,60500,1.0,1.0,1.0,1.0,1.0,2,93.0,1,0.0,2.0,25.0,11.0
4,60501,1.0,0.0,0.0,1.0,9.0,3,156.0,1,0.0,11.0,40.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4471,65953,0.0,0.0,1.0,0.0,0.0,3,0.0,1,0.0,1.0,46.0,0.0
4472,65954,0.0,1.0,0.0,0.0,0.0,3,0.0,2,0.0,4.0,17.0,0.0
4473,65965,NaN,NaN,NaN,NaN,NaN,3,NaN,2,NaN,NaN,NaN,NaN
4474,65970,0.0,1.0,1.0,1.0,0.0,3,0.0,1,1.0,0.0,110.0,0.0


In [37]:
df_FUND.columns

Index(['ID_ESCOLA', 'IN_INTERNET_ALUNOS', 'IN_BANDA_LARGA', 'IN_REFEITORIO',
       'IN_ESPACO_ATIVIDADE', 'IN_ACESSO_INTERNET_COMPUTADOR',
       'TP_DEPENDENCIA', 'QT_MAT_FUND', 'TP_LOCALIZACAO', 'QT_PROF_PSICOLOGO',
       'QT_PROF_PEDAGOGIA', 'QT_TRANSP_PUBLICO', 'QT_DOC_FUND'],
      dtype='object')

In [38]:
df_FUND.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4476 entries, 0 to 4475
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ID_ESCOLA                      4476 non-null   int64  
 1   IN_INTERNET_ALUNOS             3768 non-null   float64
 2   IN_BANDA_LARGA                 3402 non-null   float64
 3   IN_REFEITORIO                  3768 non-null   float64
 4   IN_ESPACO_ATIVIDADE            3768 non-null   float64
 5   IN_ACESSO_INTERNET_COMPUTADOR  3768 non-null   float64
 6   TP_DEPENDENCIA                 4476 non-null   int64  
 7   QT_MAT_FUND                    3759 non-null   float64
 8   TP_LOCALIZACAO                 4476 non-null   int64  
 9   QT_PROF_PSICOLOGO              3768 non-null   float64
 10  QT_PROF_PEDAGOGIA              3768 non-null   float64
 11  QT_TRANSP_PUBLICO              3759 non-null   float64
 12  QT_DOC_FUND                    3759 non-null   f

In [39]:
import numpy as np
import pandas as pd

def calculate_metrics(df,
                      pesos=None,
                      target_doc_per_100=5.0,   # 1 professor p/20 alunos
                      rural_code=2,             # ajuste conforme seu código
                      use_equity=True,
                      penalize_missing=True):
    df = df.copy()

    # --- garantir binários 0/1 (preserva NaN) ---
    bin_conn = ["IN_INTERNET_ALUNOS","IN_BANDA_LARGA","IN_ACESSO_INTERNET_COMPUTADOR"]
    bin_ambi = ["IN_REFEITORIO","IN_ESPACO_ATIVIDADE"]
    for c in bin_conn + bin_ambi:
        df[c] = np.where(df[c].isna(), np.nan, (df[c].astype(float) > 0).astype(float))

    # TRANS binário a partir de QT_TRANSP_PUBLICO
    df["TRANS"] = np.where(df["QT_TRANSP_PUBLICO"].isna(), np.nan,
                           (df["QT_TRANSP_PUBLICO"].astype(float) > 0).astype(float))

    # evitar div/0 nas razões
    df["QT_MAT_FUND"] = df["QT_MAT_FUND"].replace(0, np.nan)

    # --- sub-escores ---
    CONN = df[bin_conn].mean(axis=1, skipna=True)
    AMBI = df[bin_ambi].mean(axis=1, skipna=True)

    doc_per_100 = 100 * df["QT_DOC_FUND"] / df["QT_MAT_FUND"]
    DOC = np.clip(doc_per_100 / target_doc_per_100, 0, 1)

    apoio_per_100 = 100 * (df["QT_PROF_PSICOLOGO"].fillna(0) + df["QT_PROF_PEDAGOGIA"].fillna(0)) / df["QT_MAT_FUND"]
    p5, p95 = apoio_per_100.quantile([0.05, 0.95])
    apoio_w = apoio_per_100.clip(p5, p95)
    den = (p95 - p5) if np.isfinite(p95 - p5) and (p95 - p5) != 0 else np.nan
    APOIO = ((apoio_w - p5) / den).where(np.isfinite((apoio_w - p5) / den))

    # anexar colunas
    df["CONN"], df["AMBI"], df["DOC"], df["APOIO"] = CONN, AMBI, DOC, APOIO

    # --- pesos (human + transporte priorizados) ---
    if pesos is None:
        pesos = pd.Series({"DOC":0.35, "APOIO":0.20, "TRANS":0.20, "CONN":0.15, "AMBI":0.10})
    else:
        pesos = pd.Series(pesos)

    subs = ["DOC","APOIO","TRANS","CONN","AMBI"]
    disp = df[subs].notna()
    peso_disp = disp.mul(pesos, axis=1)
    peso_norm = peso_disp.div(peso_disp.sum(axis=1), axis=0)
    score_0a1 = (df[subs].fillna(0) * peso_norm).sum(axis=1)

    # --- multiplicador de equidade ---
    mult = 1.10 if use_equity else 1.00
    if use_equity and "TP_LOCALIZACAO" in df.columns:
        mult = np.where(df["TP_LOCALIZACAO"] == rural_code, 1.10, 1.00)

    # --- penalização por dados faltantes ---
    if penalize_missing:
        missing_share = 1 - disp.mean(axis=1)
        penal = (15 * missing_share).clip(0, 15)
    else:
        penal = 0.0

    # --- score final (0–100) ---
    df["score_infraestrutura"] = (100 * score_0a1) * mult - penal
    df["score_infraestrutura"] = df["score_infraestrutura"].clip(0, 100)

    # parâmetros usados (útil p/ reprodutibilidade)
    params = {
        "APOIO_p5": float(p5) if pd.notna(p5) else None,
        "APOIO_p95": float(p95) if pd.notna(p95) else None,
        "target_doc_per_100": float(target_doc_per_100),
        "pesos": pesos.to_dict()
    }

    final_df = df[subs + ["score_infraestrutura"]]

    return final_df, params


In [40]:
df_FUND_com_score, params = calculate_metrics(df_FUND)

In [41]:
df_FUND_com_score

,DOC,APOIO,TRANS,CONN,AMBI,score_infraestrutura
0,1.0,0.035556,1.0,0.666667,0.0,65.711111
1,NaN,NaN,NaN,NaN,NaN,0.000000
2,NaN,NaN,1.0,0.666667,1.0,82.888889
3,1.0,0.172043,1.0,1.000000,1.0,83.440860
4,1.0,0.564103,1.0,0.666667,0.5,81.282051
...,...,...,...,...,...,...
4471,NaN,NaN,1.0,0.000000,0.5,49.555556
4472,NaN,NaN,1.0,0.333333,0.0,55.111111
4473,NaN,NaN,NaN,NaN,NaN,0.000000
4474,NaN,NaN,1.0,0.333333,1.0,71.777778


In [42]:
df_MED_com_score, params_MED = calculate_metrics(df_MED)

KeyError: 'QT_MAT_FUND'